In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
properties = pd.read_csv("properties_2016.csv")
train = pd.read_csv("train_2016_v2.csv")

/anaconda/envs/Kaggle/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
properties.shape

(2985217, 58)

In [4]:
properties.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [5]:
cat_cols=[]
for col in properties.columns:
    if properties[col].dtype == object:
        cat_cols.append(col)
        print (col)

hashottuborspa
propertycountylandusecode
propertyzoningdesc
fireplaceflag
taxdelinquencyflag


In [6]:
properties = properties.drop(cat_cols, axis=1)

In [7]:
# drop the transaction date
train = train.drop(["transactiondate"], axis=1)

In [8]:
# no duplicated rows
np.where(properties.duplicated(subset='parcelid'))

(array([], dtype=int64),)

In [9]:
# duplicated rows
duplicated_train = np.where(train.duplicated(subset='parcelid', keep=False))

In [10]:
train.iloc[duplicated_train].sort_values(by='parcelid').head()

,parcelid,logerror
10828,10736972,-0.0460
10829,10736972,-0.0367
9177,10790468,-0.0450
9178,10790468,-0.0440
2226,10796614,-0.1267


In [11]:
train = train.drop_duplicates(subset='parcelid', keep='first')

In [12]:
data = pd.merge(properties, train, how='left', on=['parcelid'])

In [13]:
data.shape

(2985217, 54)

In [14]:
data.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1948.0,1.0,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1947.0,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [15]:
df_train = data.iloc[np.where(data['logerror'].notnull())]
df_test = data.iloc[np.where(data['logerror'].isnull())]

In [16]:
X_train = df_train.drop(["parcelid", "logerror"], axis=1).copy()
y_train = df_train['logerror']
X_test = df_test.drop(['parcelid', 'logerror'], axis=1).copy()

In [17]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(90150, 52)
(90150,)
(2895067, 52)


In [18]:
import xgboost as xgb

print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'silent': 1
}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

/anaconda/envs/Kaggle/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)



Setting up data for XGBoost ...


In [19]:
num_boost_rounds = 242
print("\nXGBoost tuned with CV in:")
print("   https://www.kaggle.com/aharless/xgboost-without-outliers-tweak ")
print("num_boost_rounds="+str(num_boost_rounds))

# train model
print( "\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost ...")
xgb_pred = model.predict(dtest)

print( "\nXGBoost predictions:" )
print( pd.DataFrame(xgb_pred).head() )


XGBoost tuned with CV in:
   https://www.kaggle.com/aharless/xgboost-without-outliers-tweak 
num_boost_rounds=242

Training XGBoost ...

Predicting with XGBoost ...

XGBoost predictions:
          0
0  0.106495
1  0.024238
2  0.205728
3  0.315256
4  0.234527


In [21]:
xgb_pred.shape

(2895067,)

In [22]:
X_test["logerror"] = xgb_pred

In [43]:
np.where(data['logerror'].isnull())[0]

array([      0,       1,       2, ..., 2985214, 2985215, 2985216])

In [45]:
data.loc[np.where(data['logerror'].isnull())[0], 'logerror'] = xgb_pred

In [48]:
# Create submission
submission = pd.read_csv('sample_submission.csv')

In [49]:
submission.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [52]:
output_cols = submission.columns.tolist()
print(output_cols)

['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712']


In [53]:
output_cols.remove("ParcelId")
print(output_cols)

['201610', '201611', '201612', '201710', '201711', '201712']


In [55]:
for col in output_cols:
    submission[col] = data['logerror']
submission.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.106495,0.106495,0.106495,0.106495,0.106495,0.106495
1,10759547,0.024238,0.024238,0.024238,0.024238,0.024238,0.024238
2,10843547,0.205728,0.205728,0.205728,0.205728,0.205728,0.205728
3,10859147,0.315256,0.315256,0.315256,0.315256,0.315256,0.315256
4,10879947,0.234527,0.234527,0.234527,0.234527,0.234527,0.234527


In [59]:
submission.to_csv('First_xgb_submission.csv', index=False)

## Model without missing values over 50%

In [60]:
data.shape

(2985217, 54)

In [67]:
data_null_50 = data.copy()
cols = data_null_50.columns
cols.tolist().remove("logerror")


# for col in cols:
#     if sum(data_null_50[col].isnull())/len(data_null_50[col]) > 50:
#         data_null_50 = data_null_50.drop(col, axis=1)

In [76]:
def pnull(x):
    return np.sum(x.isnull())/len(x)

In [ ]:
null_50_cols = data_null_50.apply(pnull, axis=0) > 0.5